<a href="https://colab.research.google.com/github/RaviVarshaCS/HackTX2023/blob/main/lego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install rebrick

In [3]:
import rebrick as rb
import json
import pandas as pd
from urllib.request import urlopen

In [ ]:
# API init
rb.init("e45f779f125a6a6e0a3508e5726b187b", "caseyc", "Budder0100!")

# Obtain Lego Sets given aprams
print("Getting sets...")
# Change params if necessary (ex we need to run demos)
response = rb.lego.get_sets(min_year=2000, max_year=2022, min_pieces=30, max_pieces=50)
# Convert HTTPResponse into JSON
data = response.read().decode('utf-8')
json_obj = json.loads(data)

print("Building dict...")
# Splice out dict to become Dict{"set_num: xxx" {name, num_parts, set_img_url}}
results = json_obj["results"]
df = pd.DataFrame(results)
df.set_index("set_num", inplace=True)
df = df[["name", "num_parts", "set_img_url"]]
print("dict built!")
df.head()

In [ ]:
# Add in parts list into each set_num entry

# Null entries
df["parts_list"] = [[] for _ in range(len(df))]
# Query API to get elements from each set
for set_num in df.index:
    print(set_num)
    parts = rb.lego.get_set_elements(set_num, part_details=True).read().decode('utf-8')
    parts_json = json.loads(parts)
    print(parts_json)
    parts_results = parts_json["results"]
    for part in parts_results:
      parts_dict = {'part_num': part['part']['part_num'], 'name': part['part']['name'], 'quantity':part['quantity'], 'part_img_url': part['part']['part_img_url']}
      df.at[set_num, 'parts_list'].append(parts_dict)
    # Sort each part by part num (more efficient lego set searching)
    sorted_parts_dict = sorted(df.at[set_num, 'parts_list'], key=lambda x: x['part_num'])
    df.at[set_num, 'parts_list'] = sorted_parts_dict
    print(sorted_parts_dict['part_num'])




In [71]:


result_dict = df.to_dict(orient="index")


save_file = open("set_to_set_info.json", "w")
json.dump(result_dict, save_file, indent = 4)
save_file.close()

In [57]:
df.head()

,name,num_parts,set_img_url,parts_list
set_num,,,,
10005-1,2 x 4 Sand Red Bricks,50,https://cdn.rebrickable.com/media/sets/10005-1...,"[{'part_num': {0: '3001'}, 'name': {0: 'Brick ..."
10006-1,1 x 6 Sand Red Bricks,50,https://cdn.rebrickable.com/media/sets/10006-1...,"[{'part_num': {0: '3009'}, 'name': {0: 'Brick ..."
10008-1,2 x 4 Roof Tile Sand Red,50,https://cdn.rebrickable.com/media/sets/10008-1...,"[{'part_num': {0: '3037'}, 'name': {0: 'Slope ..."
10011-1,Assorted Blue Plates,42,https://cdn.rebrickable.com/media/sets/10011-1...,"[{'part_num': {0: '4477'}, 'name': {0: 'Plate ..."
10012-1,Assorted Yellow Plates,42,https://cdn.rebrickable.com/media/sets/10012-1...,"[{'part_num': {0: '4477'}, 'name': {0: 'Plate ..."
